# Google Search Tool with Strands Agents SDK by Markus Bestehorn (bestem@amazon.com)

**Building Intelligent Web-Enabled AI Agents**

---

This comprehensive notebook demonstrates how to create a powerful Google search tool for the Strands Agents SDK and implement intelligent agents that can search the web to answer current questions, gather real-time information, and perform research tasks.

### 🎯 What You'll Build

By the end of this tutorial, you'll have:
- **A functional Google Search tool** integrated with Strands Agents
- **An intelligent research agent** that can search the web and synthesize information
- **Real-world examples** demonstrating web search capabilities
- **Performance analysis** and optimization techniques
- **Error handling** for robust production deployment

### 🚀 Key Features

- **Real-time web search** using Google Custom Search API
- **Intelligent result formatting** with titles, URLs, and descriptions
- **Configurable result limits** (1-10 results per query)
- **Robust error handling** for API failures and edge cases
- **Performance monitoring** and optimization

## Prerequisites

### 📋 Required Setup

Before starting, ensure you have:

- **✅ Google Custom Search API key** stored in `google-api-key.txt`
- **✅ Google Custom Search Engine ID** (programmable search engine)
- **✅ Strands Agents SDK** installed
- **✅ AWS credentials** configured for Bedrock access

### 📚 Documentation References

For detailed setup information:
- [Google Custom Search API Documentation](https://developers.google.com/custom-search/v1/introduction)
- [Amazon Bedrock with Strands Agents Guide](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/)

---

## 🔑 Step 1: Getting Your Google Custom Search API Key

### Overview
The Google Custom Search Engine (CSE) API allows you to programmatically search the web. You'll need a Google account to obtain an API key.

### 📝 Step-by-Step Instructions

1. **Create/Access Google Account**
   - Visit [Google Account Sign-in](https://accounts.google.com/signin)
   - Create a new account or sign in to existing one

2. **Generate API Key**
   - Navigate to: [Google Custom Search API Introduction](https://developers.google.com/custom-search/v1/introduction)
   - Click the **"Get a Key"** button in the middle of the screen
   - Enter a project name (e.g., "StrandsGoogleSearchTool")
   - Wait for key generation (takes a few seconds)
   - Click **"Show Key"** and copy the API key

3. **Secure Storage**
   - Create a new text file on your system
   - Paste the API key into the file
   - Save as `google-api-key.txt` in your working directory
   - Update the path in the cell below if needed

### 🔒 Security Best Practice
**Never store API keys directly in notebooks or code repositories!** Always use separate files and ensure they're included in your `.gitignore` file.

In [ ]:
## 📁 Configure the path to your Google API Key file
GOOGLE_API_KEY_FILE = "./google-api-key.txt"

---

## 🎯 Step 2: Creating Your Custom Search Engine ID

### Overview
The Custom Search Engine (CSE) ID defines the scope and parameters of your searches. For this tutorial, we'll create a general web search engine, but you can customize it to search specific sites, include images, or apply other filters.

### 🛠️ Configuration Options

- **🌐 Whole Web Search**: Search the entire internet (recommended for this tutorial)
- **📱 Specific Sites**: Limit searches to particular websites (e.g., Wikipedia only)
- **🖼️ Media Types**: Include images, videos, or text-only results
- **🔍 Advanced Filters**: Date ranges, language preferences, etc.

### 📝 Step-by-Step Instructions

1. **Access CSE Control Panel**
   - Ensure you're logged into your Google account
   - Visit: [Programmable Search Engine Creation](https://programmablesearchengine.google.com/controlpanel/create)

2. **Configure Search Engine**
   - Select **"Search the entire web"** option
   - Keep all other settings as default
   - Complete the CAPTCHA verification
   - Click **"Create"** at the bottom

3. **Retrieve Engine ID**
   - After creation, you'll see a confirmation page
   - Navigate to: [All Search Engines](https://programmablesearchengine.google.com/controlpanel/all)
   - Click on your newly created search engine
   - Find the **"Search Engine ID"** field
   - Copy the ID to your clipboard

4. **Save Configuration**
   - Create a new text file
   - Paste the Search Engine ID
   - Save as `google-search_engine-id.txt`
   - Update the path below if needed

In [ ]:
## 📁 Configure the path to your Search Engine ID file
SEARCH_ENGINE_ID_FILE = "./google-search_engine-id.txt"

---

## 📦 Step 3: Install Required Dependencies

### Package Overview

This demo requires several Python packages:

- **`strands-agents`**: Core Strands Agents framework
- **`strands-agents-tools`**: Additional tools and utilities
- **`strands-agents-builder`**: Agent building and configuration tools
- **`google-api-python-client`**: Google API client for search functionality

### 💡 Installation Notes
- Run this cell **once per environment**
- The `-q` flag suppresses verbose installation output
- Installation typically takes 30-60 seconds depending on your internet connection

In [ ]:
# 📦 Install required packages
!pip install strands-agents strands-agents-tools strands-agents-builder google-api-python-client -q
print("✅ All packages installed successfully!")

---

## 📚 Step 4: Import Required Libraries

### Library Overview

Here we import all the necessary libraries for our Google search tool:

- **`boto3`**: AWS SDK for Bedrock integration
- **`googleapiclient`**: Google API client for search operations
- **`strands`**: Core Strands framework components
- **`typing`**: Type hints for better code documentation

In [ ]:
# 📚 Import required libraries
import boto3
import os
import time
from googleapiclient.discovery import build
from strands import Agent, tool
from strands.models import BedrockModel
import json
from typing import List, Dict

print("✅ All libraries imported successfully!")

---

## 🔐 Step 5: Load API Credentials

### Security Overview
This section loads your API credentials from external files, following security best practices by keeping sensitive information separate from your code.

### 🔍 What This Does
- Attempts to load Google API key from file
- Attempts to load Search Engine ID from file
- Provides clear error messages if files are missing
- Sets up global variables for use throughout the notebook

In [ ]:
# 🔐 Load Google API credentials

# Load Google API key from file
try:
    with open(GOOGLE_API_KEY_FILE, 'r') as f:
        google_api_key = f.read().strip()
    print("✅ Google API key loaded successfully!")
except FileNotFoundError:
    print(f"❌ Error: {GOOGLE_API_KEY_FILE} file not found.")
    print("   Please create this file as described in the setup instructions above.")
    google_api_key = None

# Load Search Engine ID from file
try:
    with open(SEARCH_ENGINE_ID_FILE, 'r') as f:
        search_engine_id = f.read().strip()
    print("✅ Search Engine ID loaded successfully!")
except FileNotFoundError:
    print(f"❌ Error: {SEARCH_ENGINE_ID_FILE} file not found.")
    print("   Please create this file as described in the setup instructions above.")
    search_engine_id = None

# Validate credentials
if google_api_key and search_engine_id:
    print("\n🎉 All credentials loaded successfully! Ready to proceed.")
else:
    print("\n⚠️  Missing credentials. Please complete the setup steps above before continuing.")

---

## ⚙️ Step 6: Configure AWS Bedrock Model

### Overview
This section sets up the AWS Bedrock model that will power our intelligent search agent. We're using Claude 3.7 Sonnet for its excellent reasoning and text generation capabilities.

### 🔧 Configuration Options
- **Profile**: AWS CLI profile to use (change if needed)
- **Model**: Claude 3.7 Sonnet (latest available)
- **Region**: Automatically determined from your AWS configuration

### 💡 Customization
If you want to use a different AWS profile, change the `profile_name` parameter below.

In [ ]:
# ⚙️ Setup AWS session and Bedrock model
session = boto3.Session(
    profile_name='default'  # 🔧 Change this if you use a different AWS profile
)

# Create a Bedrock model with the custom session
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    boto_session=session
)

print("✅ Bedrock model configured successfully!")
print(f"   AWS Profile: {session.profile_name}")

---

## 🔧 Step 7: Create the Google Search Tool

### Tool Overview
This is the core of our implementation - a custom tool that integrates Google Custom Search with the Strands framework.

### 🚀 Key Features
- **Configurable Results**: Return 1-10 search results (default: 3)
- **Rich Formatting**: Includes title, URL, and description for each result
- **Error Handling**: Graceful handling of API failures and edge cases
- **Type Safety**: Proper type hints for reliable agent integration

### 🔍 How It Works
1. Validates API credentials are available
2. Builds Google Custom Search service
3. Executes search query with specified parameters
4. Formats results into readable text
5. Returns structured search results to the agent

In [ ]:
# 🔧 Create the Google Search Tool

@tool
def google_search(query: str, num_results: int = 3) -> str:
    """
    Search Google and return top results with titles, URLs, and snippets.
    
    Args:
        query: The search query string
        num_results: Number of results to return (default: 3, max: 10)
    
    Returns:
        Formatted string with search results including titles, URLs, and descriptions
    """
    # Validate API credentials
    if not google_api_key:
        return f"❌ Error: Google API key not available. Please check {GOOGLE_API_KEY_FILE} file."

    if not search_engine_id:
        return f"❌ Error: Search Engine ID not available. Please check {SEARCH_ENGINE_ID_FILE} file."
    
    # Validate and limit number of results
    num_results = max(1, min(num_results, 10))  # Ensure between 1-10 results
    
    try:
        # Build the Custom Search service
        service = build("customsearch", "v1", developerKey=google_api_key)
        
        # Perform the search
        result = service.cse().list(
            q=query,
            cx=search_engine_id,
            num=num_results
        ).execute()
        
        # Check if results were found
        if 'items' not in result:
            return f"🔍 No search results found for query: '{query}'"
        
        # Format the results
        formatted_results = []
        for i, item in enumerate(result['items'][:num_results], 1):
            title = item.get('title', 'No title available')
            link = item.get('link', 'No URL available')
            snippet = item.get('snippet', 'No description available')
            
            formatted_result = f"**Result {i}:**\n"
            formatted_result += f"📋 Title: {title}\n"
            formatted_result += f"🔗 URL: {link}\n"
            formatted_result += f"📝 Description: {snippet}\n"
            
            formatted_results.append(formatted_result)
        
        return "\n".join(formatted_results)
        
    except Exception as e:
        return f"❌ Error performing Google search: {str(e)}"

print("✅ Google search tool created successfully!")
print("   Features: Configurable results (1-10), rich formatting, error handling")

---

## 🧪 Step 8: Test the Search Tool Directly

### Testing Overview
Before integrating with an agent, let's test our search tool directly to ensure it's working correctly.

### 🔍 What This Test Does
- Executes a sample search query
- Displays formatted search results
- Validates that the API integration is working
- Shows the structure of returned data

In [ ]:
# 🧪 Test the search tool directly
test_query = "artificial intelligence latest news 2024"
print(f"🔍 Testing search tool with query: '{test_query}'")
print("=" * 60)

search_results = google_search(test_query)
print(search_results)

print("\n" + "=" * 60)
print("✅ Direct search tool test completed!")

---

## 🤖 Step 9: Create the Search Agent

### Agent Overview
Now we'll create an intelligent agent that can use our Google search tool to answer questions requiring current information or web research.

### 🧠 Agent Capabilities
- **Intelligent Decision Making**: Determines when web search is needed
- **Source Citation**: Always provides URLs and references
- **Information Synthesis**: Combines multiple search results into coherent answers
- **Contextual Responses**: Tailors responses to the user's specific question

### 📝 System Prompt Design
The system prompt instructs the agent on:
- When to use the search tool
- How to cite sources properly
- Response formatting and style
- Information accuracy and reliability

In [ ]:
# 🤖 Create an intelligent search agent
search_agent = Agent(
    system_prompt="""You are a helpful research assistant equipped with Google search capabilities. 
    
    🔍 WHEN TO SEARCH:
    - Use the google_search tool when users ask questions that require:
      • Current information or recent news
      • Facts that you might not know or that may have changed
      • Real-time data (weather, stock prices, etc.)
      • Comparative information about products, services, or events
    
    📋 RESPONSE GUIDELINES:
    - Always cite your sources with URLs when presenting search results
    - Be concise but informative in your responses
    - Synthesize information from multiple sources when relevant
    - Indicate when information might be time-sensitive
    - If search results are limited or unclear, acknowledge this limitation
    
    ✅ QUALITY STANDARDS:
    - Prioritize authoritative and recent sources
    - Present information objectively
    - Clearly distinguish between facts and opinions
    - Provide context when necessary for understanding""",
    tools=[google_search],
    model=bedrock_model,
)

print("✅ Search agent created successfully!")
print("   Capabilities: Web search, source citation, information synthesis")
print("   Model: Claude 3.7 Sonnet")
print("   Tools: Google Custom Search")

---

## 🎯 Step 10: Demo - Agent Using Google Search

### Demo Overview
Let's test our search agent with various types of questions that require web search capabilities.

### 📝 Test Questions
We'll test three different types of queries:
1. **Technology News**: Latest AI developments
2. **Real-time Data**: Current weather information
3. **Recent Events**: Nobel Prize winners

### 🔍 What to Observe
- How the agent decides when to use the search tool
- Quality of information synthesis
- Source citation and URL inclusion
- Response formatting and readability

In [ ]:
# 🎯 Test the search agent with various question types

test_questions = [
    "What are the latest developments in AI in 2024?",
    "What is the current weather in London?",
    "Who won the latest Nobel Prize in Physics?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"🔍 Test Question {i}: {question}")
    print(f"{'='*80}")
    
    # Measure response time
    start_time = time.time()
    response = search_agent(question)
    response_time = time.time() - start_time
    
    print(f"🤖 Agent's Response:")
    print(response)
    print(f"\n⏱️  Response Time: {response_time:.2f} seconds")

print(f"\n{'='*80}")
print("✅ All test questions completed successfully!")

---

## 🔬 Step 11: Advanced Example - Comparative Research

### Advanced Use Case
This section demonstrates how the search agent can handle complex comparative research questions that require multiple searches and information synthesis.

### 🎯 Research Scenario
We'll ask the agent to compare two competing products, which typically requires:
- Multiple search queries
- Information from different sources
- Objective comparison and analysis
- Clear presentation of findings

In [ ]:
# 🔬 Advanced research example
research_question = "Compare the latest iPhone vs Samsung Galaxy phones released in 2024"

print(f"📊 Research Question: {research_question}")
print("=" * 100)

# Measure comprehensive research time
start_time = time.time()
response = search_agent(research_question)
research_time = time.time() - start_time

print(f"🔍 Research Assistant's Response:")
print(response)
print(f"\n⏱️  Total Research Time: {research_time:.2f} seconds")
print("\n" + "=" * 100)
print("✅ Comparative research completed!")

---

## 📊 Step 12: Performance Analysis

### Performance Overview
Understanding the performance characteristics of your search tool is crucial for production deployment.

### 📈 Metrics We'll Measure
- **Direct Tool Call Time**: Raw search API performance
- **Agent Processing Time**: Total time including AI reasoning
- **AI Processing Overhead**: Additional time for intelligent processing

### 🎯 Optimization Insights
This analysis helps identify:
- Bottlenecks in the search process
- Trade-offs between speed and intelligence
- Opportunities for caching and optimization

In [ ]:
# 📊 Performance analysis function
def analyze_search_performance(query: str):
    """Comprehensive performance analysis of our Google search tool"""
    print(f"📊 Analyzing performance for query: '{query}'")
    print("-" * 60)
    
    # Test direct tool performance
    print("🔧 Testing direct tool call...")
    start_time = time.time()
    direct_result = google_search(query)
    direct_time = time.time() - start_time
    
    # Test agent performance (includes AI processing)
    print("🤖 Testing agent with tool...")
    start_time = time.time()
    agent_result = search_agent(f"Search for: {query}")
    agent_time = time.time() - start_time
    
    # Calculate metrics
    ai_overhead = agent_time - direct_time
    efficiency_ratio = direct_time / agent_time * 100
    
    # Display results
    print(f"\n📊 Performance Results:")
    print(f"   🔧 Direct tool call: {direct_time:.2f} seconds")
    print(f"   🤖 Agent with tool: {agent_time:.2f} seconds")
    print(f"   ⚡ AI processing overhead: {ai_overhead:.2f} seconds")
    print(f"   📈 Efficiency ratio: {efficiency_ratio:.1f}%")
    
    return direct_time, agent_time

# Run performance analysis
print("🚀 Running performance analysis...")
analyze_search_performance("Python programming best practices")

---

## 🧪 Step 13: Error Handling and Edge Cases

### Robustness Testing
Testing how our search agent handles various edge cases and error conditions is crucial for production reliability.

### 🔍 Test Cases
We'll test several challenging scenarios:
- **Empty queries**: How does the system handle blank input?
- **Nonsensical queries**: What happens with random text?
- **No-result queries**: How are failed searches handled?

### 🛡️ Error Handling Benefits
- Prevents system crashes
- Provides meaningful feedback to users
- Maintains consistent user experience
- Enables graceful degradation

In [ ]:
# 🧪 Test error handling and edge cases

edge_cases = [
    ("", "Empty query"),
    ("asdfghjklqwertyuiopzxcvbnm123456789", "Nonsensical query"),
    ("site:nonexistentwebsite12345.com information", "No results expected")
]

print("🧪 Testing Edge Cases and Error Handling")
print("=" * 80)

for i, (test_case, description) in enumerate(edge_cases, 1):
    print(f"\n🔍 Test Case {i}: {description}")
    print(f"   Query: '{test_case}'")
    print("-" * 50)
    
    try:
        if test_case:  # Only test non-empty strings with the agent
            response = search_agent(f"Please search for: {test_case}")
            print(f"✅ Agent Response: {response[:200]}{'...' if len(response) > 200 else ''}")
        else:
            # Test empty query directly with the tool
            direct_response = google_search(test_case)
            print(f"🔧 Direct Tool Response: {direct_response}")
    except Exception as e:
        print(f"❌ Error caught: {e}")

print("\n" + "=" * 80)
print("✅ Edge case testing completed successfully!")

---

## 📋 Summary and Next Steps

### 🎉 What We've Accomplished

This notebook has successfully demonstrated:

#### 1. **🔧 Google Search Tool Creation**
- Built a custom `@tool` function that integrates with Google Custom Search API
- Implemented proper error handling and input validation
- Added configurable result limits and rich formatting

#### 2. **🤖 Intelligent Agent Integration**
- Created an agent that can intelligently decide when to use web search
- Implemented proper source citation and URL referencing
- Designed contextual response generation

#### 3. **🎯 Real-world Testing**
- Tested with various query types including current events and comparative research
- Demonstrated complex research scenarios
- Validated performance across different use cases

#### 4. **📊 Performance Analysis**
- Measured tool and agent response times
- Identified AI processing overhead
- Provided optimization insights

#### 5. **🛡️ Error Handling**
- Demonstrated graceful handling of edge cases and errors
- Tested system robustness with various input scenarios
- Ensured reliable production-ready behavior

---

### 🚀 Key Features of Our Google Search Tool

- **📊 Configurable Results**: Returns 1-10 search results (default: 3)
- **🎨 Rich Formatting**: Includes title, URL, and description for each result
- **🛡️ Robust Error Handling**: Graceful handling of API failures and edge cases
- **🔍 Type Safety**: Proper type hints for reliable agent integration
- **⚡ Performance Optimized**: Efficient API usage with result limiting
- **🔐 Security Focused**: External credential storage and validation

---

### 🔮 Potential Enhancements

Consider these improvements for advanced implementations:

#### **🎯 Search Capabilities**
- Add support for different search types (images, news, videos)
- Implement advanced search operators and filters
- Add date range and language filtering options
- Support for site-specific searches

#### **⚡ Performance Optimizations**
- Implement intelligent caching to avoid duplicate API calls
- Add result summarization for large result sets
- Implement parallel search processing for multiple queries
- Add response compression and optimization

#### **🔧 Integration Features**
- Add support for multiple search engines (Bing, DuckDuckGo)
- Implement search result ranking and relevance scoring
- Add content extraction and full-text analysis
- Support for multimedia search results

#### **🛡️ Production Features**
- Implement comprehensive logging and monitoring
- Add rate limiting and quota management
- Implement content filtering and safety checks
- Add usage analytics and reporting

---

### 🏭 Production Considerations

When deploying this solution in production:

#### **🔐 Security**
- Implement proper API key rotation and management
- Use AWS Secrets Manager or similar for credential storage
- Add input sanitization and validation
- Implement access logging and audit trails

#### **📊 Monitoring**
- Set up comprehensive logging for all search operations
- Monitor API usage and quota consumption
- Track response times and error rates
- Implement alerting for system issues

#### **⚡ Performance**
- Implement intelligent caching strategies
- Use connection pooling for API calls
- Add load balancing for high-traffic scenarios
- Monitor and optimize query performance

#### **🔧 Reliability**
- Implement circuit breakers for API failures
- Add retry logic with exponential backoff
- Create fallback mechanisms for service outages
- Implement health checks and monitoring

---

### 🎓 Learning Resources

To continue your journey with Strands Agents and web search integration:

- **[Strands Agents Documentation](https://strandsagents.com/0.1.x/)**
- **[Google Custom Search API Guide](https://developers.google.com/custom-search/v1/introduction)**
- **[AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)**
- **[Python Type Hints Guide](https://docs.python.org/3/library/typing.html)**

---

### 🎉 Congratulations!

You've successfully built a powerful, intelligent web search agent using the Strands framework. Your agent can now:

- 🔍 **Search the web** for current information
- 🧠 **Make intelligent decisions** about when to search
- 📚 **Synthesize information** from multiple sources
- 🎯 **Provide accurate, cited responses** with source URLs
- 🛡️ **Handle errors gracefully** in production environments

Happy searching with your Strands Agent! 🔍🤖✨